# Advanced Querying Mongo

Importing libraries and setting up connection

In [125]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")

In [126]:
client.list_database_names()

['Quotes',
 'Species',
 'admin',
 'companies',
 'config',
 'final',
 'geo',
 'labflask',
 'local']

In [127]:
db = client.get_database("companies")
db.list_collection_names()

['companies', 'offices']

In [128]:
c = db.get_collection("companies")

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [16]:
condicion = {"name":"Babelgum"}
proj = {"name":1}

list(c.find(condicion, proj))

[{'_id': ObjectId('52cdef7c4bab8bd675297da0'), 'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [113]:
condicion = {"number_of_employees": {"$gt":5000}}
proj = {"name":1, "number_of_employees":1}

list(c.find(condicion,proj).sort("number_of_employees").limit(20))

[{'_id': ObjectId('52cdef7c4bab8bd675297e49'),
  'name': 'Nintendo',
  'number_of_employees': 5080},
 {'_id': ObjectId('52cdef7e4bab8bd67529ba2e'),
  'name': 'Hexaware Technologies',
  'number_of_employees': 5200},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'number_of_employees': 5299},
 {'_id': ObjectId('52cdef7e4bab8bd67529a453'),
  'name': 'OpenText',
  'number_of_employees': 5300},
 {'_id': ObjectId('52cdef7e4bab8bd67529b9dd'),
  'name': 'CPM Braxis',
  'number_of_employees': 5400},
 {'_id': ObjectId('52cdef7d4bab8bd675298b89'),
  'name': 'LSI',
  'number_of_employees': 5400},
 {'_id': ObjectId('52cdef7e4bab8bd67529b691'),
  'name': 'Microchip Technologies',
  'number_of_employees': 5500},
 {'_id': ObjectId('52cdef7e4bab8bd67529a84c'),
  'name': 'Mediaset',
  'number_of_employees': 5729},
 {'_id': ObjectId('52cdef7d4bab8bd675299f38'),
  'name': 'Mindray Medical International',
  'number_of_employees': 5763},
 {'_id': ObjectId('52cdef7d4bab8bd675298b6f'),

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [33]:
cond1 = {"founded_year":{"$gte":2000}}
cond2 = {"founded_year":{"$lte":2005}}
proj = {"name":1, "founded_year":1}

list(c.find({"$and": [cond1,cond2]}, proj).limit(3))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'founded_year': 2005},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8c'),
  'name': 'Zoho',
  'founded_year': 2005},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8d'),
  'name': 'Digg',
  'founded_year': 2004}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [34]:
q1 = {"ipo.valuation_amount":{"$gt":100000000}}
q2 = {"founded_year":{"$lte":2010}}
proj = {"name":1, "ipo":1}

list(c.find({"$and": [q1, q2]}, proj).limit(3))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d94'),
  'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297de0'),
  'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [85]:
# Utilizo el proj para que solo me de los nombres y no me saque todos los datos.
proj = {"name":1,"number_of_employees":1}

q1 = {"number_of_employees":{"$lt":1000}}
q2 = {"founded_year":{"$lte":2005}}

list(c.find({"$and": [q1, q2]}, proj).sort("number_of_employees").limit(10))

[{'_id': ObjectId('52cdef7c4bab8bd675297d93'),
  'name': 'Fox Interactive Media',
  'number_of_employees': 0},
 {'_id': ObjectId('52cdef7c4bab8bd675297e12'),
  'name': 'Ticketmaster',
  'number_of_employees': 0},
 {'_id': ObjectId('52cdef7c4bab8bd675297eba'),
  'name': 'MindTouch',
  'number_of_employees': 0},
 {'_id': ObjectId('52cdef7c4bab8bd675297dbc'),
  'name': 'Skype',
  'number_of_employees': 0},
 {'_id': ObjectId('52cdef7c4bab8bd675297dbd'),
  'name': 'YouTube',
  'number_of_employees': 0},
 {'_id': ObjectId('52cdef7c4bab8bd675297e2e'),
  'name': 'stylediary',
  'number_of_employees': 0},
 {'_id': ObjectId('52cdef7c4bab8bd675297f89'),
  'name': 'Attributor',
  'number_of_employees': 0},
 {'_id': ObjectId('52cdef7c4bab8bd675297f20'),
  'name': 'Yeigo',
  'number_of_employees': 0},
 {'_id': ObjectId('52cdef7c4bab8bd675297ee8'),
  'name': 'PartnerUp',
  'number_of_employees': 0},
 {'_id': ObjectId('52cdef7c4bab8bd675297df7'),
  'name': 'Bebo',
  'number_of_employees': 0}]

### 6. All the companies that don't include the `partners` field.

In [82]:
proj = {"name":1}

list(c.find({"partners":{"$exists":True}}, proj).limit(3))
# Utilizo el proj para que solo me de los nombres y no me saque todos los datos.

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'), 'name': 'Wetpaint'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8b'), 'name': 'AdventNet'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8c'), 'name': 'Zoho'}]

### 7. All the companies that have a null type of value on the `category_code` field.

In [133]:
q1 = {"category_code":{"$ne":"null"}}


list(c.find(q1).limit(3))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'permalink': 'abc2',
  'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
  'homepage_url': 'http://wetpaint-inc.com',
  'blog_url': 'http://digitalquarters.net/',
  'blog_feed_url': 'http://digitalquarters.net/feed/',
  'twitter_username': 'BachelrWetpaint',
  'category_code': 'web',
  'number_of_employees': 47,
  'founded_year': 2005,
  'founded_month': 10,
  'founded_day': 17,
  'deadpooled_year': 1,
  'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
  'alias_list': '',
  'email_address': 'info@wetpaint.com',
  'phone_number': '206.859.6300',
  'description': 'Technology Platform Company',
  'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
  'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
  'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and mon

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [80]:
q1 = {"number_of_employees":{"$gte":100}}
q2 = {"number_of_employees":{"$lte":1000}}
proj = {"name":1,"number_of_employees":1}

list(c.find({"$and": [q1, q2]}, proj).sort("number_of_employees").limit(3))
# Utilizo el proj para que solo me de los nombres y no me saque todos los datos.

[{'_id': ObjectId('52cdef7c4bab8bd675297e44'),
  'name': 'hi5',
  'number_of_employees': 100},
 {'_id': ObjectId('52cdef7c4bab8bd675297e27'),
  'name': 'Simply Hired',
  'number_of_employees': 100},
 {'_id': ObjectId('52cdef7c4bab8bd675297e19'),
  'name': 'Redfin',
  'number_of_employees': 100}]

### 9. Order all the companies by their IPO price in a descending order.

In [79]:
list(c.find({"ipo.price":{"$exists":True}}).sort("ipo.price", -1).limit(3))
# Utilizo el proj para que solo me de los nombres y no me saque todos los datos.

[]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [78]:
list(c.find({"number_of_employees":{"$exists":True}},{"name":1}).sort("number_of_employees").limit(3))
# Utilizo el proj para que solo me de los nombres y no me saque todos los datos.

[{'_id': ObjectId('52cdef7c4bab8bd675297d92'), 'name': 'Flektor'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8f'), 'name': 'Omnidrive'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d90'), 'name': 'Postini'}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [77]:
q1 = {"founded_month":{"$gte":6}}
proj = {"name":1}

list(c.find(q1,proj).limit(3))
# Utilizo el proj para que solo me de los nombres y no me saque todos los datos.

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'), 'name': 'Wetpaint'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8c'), 'name': 'Zoho'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8d'), 'name': 'Digg'}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [76]:
q1 = {"founded_year":{"$lt":2000}}
q2 = {"acquisition.price_amount":{"$gt":10000000}}
proj = {"name":1}

list(c.find({"$and": [q1, q2]},proj).limit(3))
# Utilizo el proj para que solo me de los nombres y no me saque todos los datos.

[{'_id': ObjectId('52cdef7c4bab8bd675297d90'), 'name': 'Postini'},
 {'_id': ObjectId('52cdef7c4bab8bd675297deb'), 'name': 'SideStep'},
 {'_id': ObjectId('52cdef7c4bab8bd675297e2c'), 'name': 'Recipezaar'}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [118]:
q1 = {"acquisition.acquired_year":{"$gt":2010}}
proj = {"name":1,"acquisition":1}

list(c.find(q1,proj).sort("acquisition.price_amount").limit(3)) 
# Utilizo el proj para que solo me de los nombres y no me saque todos los datos.

[{'_id': ObjectId('52cdef7c4bab8bd675297dbf'),
  'name': 'blogTV',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2013/03/13/younow-buys-blogtv/',
   'source_description': 'Live Social Video Network YouNow Acquires Streaming Service BlogTV',
   'acquired_year': 2013,
   'acquired_month': 3,
   'acquired_day': 13,
   'acquiring_company': {'name': 'YouNow', 'permalink': 'younow'}}},
 {'_id': ObjectId('52cdef7c4bab8bd675297dab'),
  'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [92]:
q1 = {"founded_year":{"$exists":True}}
proj = {"name":1, "founded_year":1}

list(c.find(q1, proj).sort("founded_year", -1).limit(3))

[{'_id': ObjectId('52cdef7c4bab8bd6752982d4'),
  'name': 'Advaliant',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7c4bab8bd67529801f'),
  'name': 'Wamba',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7c4bab8bd675297fec'),
  'name': 'Fixya',
  'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [94]:
q1 = {"founded_day":{"$lte":7}}
proj = {"name":1, "founded_day":1, "acquisition.price_amount":1}

list(c.find(q1, proj).sort("acquisition.price_amount", -1).limit(3))

[{'_id': ObjectId('52cdef7d4bab8bd6752989a1'),
  'name': 'Netscape',
  'founded_day': 4,
  'acquisition': {'price_amount': 4200000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297e89'),
  'name': 'PayPal',
  'founded_day': 1,
  'acquisition': {'price_amount': 1500000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297efe'),
  'name': 'Zappos',
  'founded_day': 1,
  'acquisition': {'price_amount': 1200000000}}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [101]:
q1 = {"category_code":"web"}
q2 = {"number_of_employees":{"$gt":4000}}
proj = {"name":1,"homepage_url":1, "number_of_employees":1}

list(c.find({"$and": [q1, q2]},proj).sort("number_of_employees").limit(3))

[{'_id': ObjectId('52cdef7c4bab8bd67529822a'),
  'name': 'Expedia',
  'homepage_url': 'http://www.expedia.com',
  'number_of_employees': 4400},
 {'_id': ObjectId('52cdef7c4bab8bd675297e96'),
  'name': 'AOL',
  'homepage_url': 'http://www.aol.com',
  'number_of_employees': 8000},
 {'_id': ObjectId('52cdef7c4bab8bd675297ea4'),
  'name': 'Webkinz',
  'homepage_url': 'http://www.webkinz.com',
  'number_of_employees': 8657}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [110]:
q1 = {"acquisition.price_amount":{"$gt":10_000_000}}
q2 = {"acquisition.price_currency_code":"EUR"}
proj = {"name":1,"acquisition.price_amount":1, "acquisition.price_currency_code":1}

list(c.find({"$and": [q1, q2]},proj).limit(3))



[{'_id': ObjectId('52cdef7c4bab8bd675297f02'),
  'name': 'ZYB',
  'acquisition': {'price_amount': 31500000, 'price_currency_code': 'EUR'}},
 {'_id': ObjectId('52cdef7d4bab8bd675298bf3'),
  'name': 'Apertio',
  'acquisition': {'price_amount': 140000000, 'price_currency_code': 'EUR'}},
 {'_id': ObjectId('52cdef7d4bab8bd675298f47'),
  'name': 'Greenfield Online',
  'acquisition': {'price_amount': 40000000, 'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [111]:
q1 = {"acquisition.acquired_month":{"$lte":4}}
proj = {"name":1,"acquisition":1}

list(c.find(q1,proj).limit(10))



[{'_id': ObjectId('52cdef7c4bab8bd675297d95'),
  'name': 'StumbleUpon',
  'acquisition': {'price_amount': 29000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2009/04/13/ebay-unacquires-stumbleupon/',
   'source_description': "StumbleUpon Beats Skype In Escaping EBay's Clutches",
   'acquired_year': 2009,
   'acquired_month': 4,
   'acquired_day': None,
   'acquiring_company': {'name': 'StumbleUpon', 'permalink': 'stumbleupon'}}},
 {'_id': ObjectId('52cdef7c4bab8bd675297dab'),
  'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink':

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [112]:
q1 = {"founded_year":{"$gt":2000}}
q1 = {"founded_year":{"$lt":2010}}
q3 = {"acquisition.acquired_year":{"$gt":2011}}
proj = {"name":1,"founded_year":1,"acquisition.acquired_year":1}

list(c.find({"$and": [q1, q2, q3]},proj).limit(3))

[{'_id': ObjectId('52cdef7d4bab8bd67529a229'),
  'name': 'Citizenside',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2013}},
 {'_id': ObjectId('52cdef7e4bab8bd67529a536'),
  'name': 'Webedia',
  'founded_year': 2007,
  'acquisition': {'acquired_year': 2013}},
 {'_id': ObjectId('52cdef7e4bab8bd67529a5ad'),
  'name': 'hybris',
  'founded_year': 1997,
  'acquisition': {'acquired_year': 2013}}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [124]:
db.companies.aggregate({"years_alive":{"$substract":["$deadpooled_year","$founded_year"]}})

TypeError: pipeline must be a list

In [122]:
q1 = {"years_alive":{"$lte":3}}
proj = {"name":1,"years_alive":1}

list(c.find(q1,proj).limit(10))

[]